# 1. 남동발전 데이터 확인

In [ ]:
import os, time, glob, calendar, datetime, shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
# === 0️⃣ 기본 설정 ===
BASE_PATH = r"C:\ESG_Project1\file\solar_data_file"
os.makedirs(BASE_PATH, exist_ok=True)

FIRST_YEAR = 2022
LAST_YEAR = 2025
MAX_RETRY = 3  # 다운로드 재시도 최대 횟수

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": BASE_PATH,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
driver.get("https://www.koenergy.kr/kosep/gv/nf/dt/nfdt21/main.do")
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "strDateS")))

# === 1️⃣ CSV 파일 이동 및 인코딩 처리 함수 ===
def move_and_encode_csv(year_dir: str, file_name: str):
    list_files = glob.glob(os.path.join(BASE_PATH, "*.csv"))
    if not list_files:
        return None

    latest_file = max(list_files, key=os.path.getmtime)
    dest_path = os.path.join(year_dir, file_name)

    # 한글 깨짐 방지: UTF-8-sig로 재저장
    try:
        with open(latest_file, 'r', encoding='cp949', errors='ignore') as f:
            content = f.read()
        with open(dest_path, 'w', encoding='utf-8-sig') as f:
            f.write(content)
    except Exception as e:
        print(f"⚠ 인코딩 변환 실패: {e}")
        shutil.copy(latest_file, dest_path)

    os.remove(latest_file)
    return dest_path

# === 2️⃣ 연도별 월별 반복 다운로드 ===
for year in range(FIRST_YEAR, LAST_YEAR + 1):
    year_dir = os.path.join(BASE_PATH, str(year))
    os.makedirs(year_dir, exist_ok=True)

    for month in range(1, 13):
        start_date = f"{year}{month:02d}01"
        last_day = calendar.monthrange(year, month)[1]  # 28~31일까지 자동 처리
        end_date = f"{year}{month:02d}{last_day:02d}"
        file_name = f"남동발전량_{year}_{month:02d}.csv"

        print(f"▶ {year}-{month:02d} ({start_date} ~ {end_date}) 조회 중...")

        success = False
        for attempt in range(1, MAX_RETRY + 1):
            try:
                # 날짜 JS 직접 설정
                driver.execute_script(f"document.getElementById('strDateS').value='{start_date}'")
                driver.execute_script(f"document.getElementById('strDateE').value='{end_date}'")

                # 조회 버튼 클릭
                search_btn = driver.find_element(By.XPATH, "//a[contains(@href, 'goSubmit') or contains(text(),'조회')]")
                search_btn.click()
                time.sleep(3)  # 데이터 로딩 대기

                # CSV 다운로드 (JS 함수 실행)
                driver.execute_script("goCsvDown();")
                time.sleep(5)  # 다운로드 안정화 대기

                # 파일 이동 및 인코딩 처리
                dest_path = move_and_encode_csv(year_dir, file_name)
                if dest_path:
                    print(f"✅ 저장 완료 → {dest_path}")
                    success = True
                    break
                else:
                    print(f"⚠ 다운로드 파일을 찾지 못했습니다. 재시도 {attempt}/{MAX_RETRY}")
                    time.sleep(2)

            except Exception as e:
                print(f"❌ {year}-{month:02d} 오류 발생: {e}. 재시도 {attempt}/{MAX_RETRY}")
                time.sleep(2)

        if not success:
            print(f"❌ {year}-{month:02d} 다운로드 실패, 다음 달로 진행합니다.")

driver.quit()
print("\n🎉 모든 CSV 파일이 연도별 폴더에 안전하게 저장되었습니다.")

▶ 2022-01 (20220101 ~ 20220131) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_01.csv
▶ 2022-02 (20220201 ~ 20220228) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_02.csv
▶ 2022-03 (20220301 ~ 20220331) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_03.csv
▶ 2022-04 (20220401 ~ 20220430) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_04.csv
▶ 2022-05 (20220501 ~ 20220531) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_05.csv
▶ 2022-06 (20220601 ~ 20220630) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_06.csv
▶ 2022-07 (20220701 ~ 20220731) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_07.csv
▶ 2022-08 (20220801 ~ 20220831) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_08.csv
▶ 2022-09 (20220901 ~ 20220930) 조회 중...
✅ 저장 완료 → C:\ESG_Project1\file\solar_data_file\2022\남동발전량_2022_09.csv
▶ 2022-10 